In [3]:
%load_ext autoreload
%autoreload
import pandas as pd
import dataWrangling
from memoryReduction import *
import gc
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score
sns.set()

In [3]:
df = dataWrangling.create_dt(is_train=True, first_day= 1890)

24
Mem. usage decreased to 41.17 Mb (26.2% reduction)


In [4]:
import multiprocessing as mp
importlib.reload(dataWrangling)
#p = mp.Pool(2)
df = dataWrangling.create_fea(df)
df = reduce_mem_usage(df)
#df1 = p.map(dataWrangling.create_fea, df)

7 lag_7


In [5]:
df.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'price_lag_7',
       'rmean_7_7', 'rstd_7_7', 'rskew_7_7', 'rskurt_7_7', 'week', 'day',
       'dayofweek'],
      dtype='object')

In [6]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + \
            ["event_name_1", "event_name_2", "event_type_1", "event_type_2"] +\
            ['year','month','week','day','dayofweek']
useless_cols = ["id", "date", "sales","d", "wm_yr_wk"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]
X_train = reduce_mem_usage(X_train)
y_train = reduce_mem_usage(y_train)

del df
gc.collect()


In [6]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,sell_price,lag_7,price_lag_7,rmean_7_7,rstd_7_7,rskew_7_7,rskurt_7_7,week,day,dayofweek
243925,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1903,1.0,2016-04-14,11611,...,8.382812,3.0,8.257812,1.428711,1.618164,0.674316,-1.151367,2,13,3
243926,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1904,1.0,2016-04-15,11611,...,8.382812,0.0,8.257812,1.428711,1.618164,0.674316,-1.151367,2,14,4
243939,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,d_1903,2.0,2016-04-14,11611,...,2.970703,1.0,2.970703,0.285645,0.488037,1.229492,-0.839844,2,13,3
243940,HOBBIES_1_003_CA_1_validation,2,0,0,0,0,d_1904,2.0,2016-04-15,11611,...,2.970703,0.0,2.970703,0.285645,0.488037,1.229492,-0.839844,2,14,4
243946,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_1903,0.0,2016-04-14,11611,...,4.640625,0.0,4.640625,1.571289,1.133789,0.235229,-1.227539,2,13,3


In [7]:
'''
train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
fake_valid_inds = np.random.choice(len(X_train), 500000)
fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
                             free_raw_data=False)



In [4]:
#taken from https://www.kaggle.com/bitit1994/parameter-grid-search-lgbm-with-scikit-learn
'''
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        'verbosity': 1,
        'num_iterations' : 25,
}
'''
# Initiate classifier to use
mdl = lgb.LGBMClassifier()

In [5]:
gridParams = {
    'learning_rate': [0.005, 0.01,0.075],
#    'n_estimators': [8,16,24],
    'num_leaves': [6,12,32,64], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['poisson','regression'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    "sub_feature" : [0.9,0.8,0.7],
    "sub_row" : [0.75,0.8,0.85],
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75,0.8],
    'lambda_l1' : [0.1,0.05,0.15,0.5,1,1.2],
    "lambda_l2" : [0.1,0.05,0.15,0.5,1,1.2],
    "metric" : ['rmse'],
    'num_iterations' : [100],
    'verbosity': [1],
    "force_row_wise" : [True],
    "bagging_freq" : [1],
    }

grid = GridSearchCV(mdl, gridParams, verbose=1, n_jobs=4,cv = 4) ##remember to add cv
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'